In [1]:
from yfpy.query import YahooFantasySportsQuery
from yfpy.models import YahooFantasyObject
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
from nhlpy import NHLClient
from yfpy.utils import reformat_json_list, unpack_data, prettify_data, jsonify_data
import requests
import json

In [2]:
load_dotenv()
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [3]:
nhl_client = NHLClient()

In [50]:
# Use last season stats so there aren't any spoilers
current = False
league_id = "67269" if current else "97108"
game_id = 453 if current else 427
query = YahooFantasySportsQuery(
    league_id=league_id,
    game_code="nhl",
    game_id=game_id,
    yahoo_consumer_key=CLIENT_ID,
    yahoo_consumer_secret=CLIENT_SECRET,
)

In [5]:
league_info = query.get_league_info()
league_start_week = league_info.start_week
league_end_week = league_info.end_week
league_start_date_str = league_info.start_date
league_end_date_str = league_info.end_date

In [7]:
def get_league_game_weeks():
    game_weeks = query.query(
        f"https://fantasysports.yahooapis.com/fantasy/v2/game/{game_id}/game_weeks", 
        ["game", "game_weeks"]
    )
    game_weeks_league = game_weeks
    game_weeks_league[league_start_week-1].start = league_start_date_str
    game_weeks_league[league_end_week-1].end = league_end_date_str
    return game_weeks_league

In [8]:
league_game_weeks = get_league_game_weeks()

In [9]:
def get_dates_by_week(week):
    start_date_str = league_game_weeks[week-1].start
    end_date_str = league_game_weeks[week-1].end
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    return [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end_date - start_date).days + 1)]

In [179]:
def get_players(player_keys):
    return [
        player 
        for i in range(int(len(player_keys) / 25) + 1)
        for player in query.query(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats",
            ["league", "players"]
        )
    ]

In [54]:
def get_top_n_players_by_position(n, position):
    if position == "F":
        position = "C,LW,RW"
    return [playere for i in range(int(n/25)+1) for player in query.query(
        f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;sort=PTS;sort_type=season;position={position};count={n};start={i*25}/stats",
        ["league", "players"]
    )]

In [55]:
'''
Get's daily scores for each matchup of the week.
'''
def get_matchups_by_week(week):
    dates = get_dates_by_week(week)
    resp = query.get_response(
        f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/scoreboard;week={week}/matchups/teams/stats_collection;types=date;date={','.join(dates)}", 
    )
    teams_dict = resp.json().get('fantasy_content', {}).get('league', [{}])[1].get('scoreboard', {}).get("0").get("matchups", {})
    teams_dict = unpack_data(teams_dict, YahooFantasyObject)
    teams_dict = [el["matchup"] for el in teams_dict] # Convert object to list
    return teams_dict

In [91]:
'''
Biggest comeback
52.8s
'''
deficits = []
for week in range(league_start_week, league_end_week + 1):
    matchups = get_matchups_by_week(week)
    for matchup in matchups:
        # Comeback win can't happen without a winner
        if matchup.is_tied:
            continue
        winner_id = int(matchup.winner_team_key.split(".")[-1])
        team_ids = [matchup.teams[0].team_id, matchup.teams[1].team_id]
        team_w_idx, team_l_idx = (0, 1) if winner_id == team_ids[0] else (1, 0)
        deficit = 0
        start_date = datetime.strptime(matchup.week_start, "%Y-%m-%d")
        end_date = datetime.strptime(matchup.week_end, "%Y-%m-%d")
        # Don't include the last day since the matchup is over
        for i in range((end_date - start_date).days):
            current_date_str = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
            team_w_points = matchup.teams[team_w_idx].team_stats_collection[i]["team_points"].total
            team_l_points = matchup.teams[team_l_idx].team_stats_collection[i]["team_points"].total
            deficit = round(deficit + team_l_points - team_w_points, 1)
            if deficit > 0:
                deficits.append((deficit, matchup))
biggest_deficits = sorted(deficits, key=lambda x: x[0])
biggest_deficits = list({f"{matchup.week}.{matchup.winner_team_key}": (deficit, matchup) for (deficit, matchup) in biggest_deficits}.values())
biggest_deficits = sorted(biggest_deficits, reverse=True, key=lambda x: x[0])
print("Biggest comebacks")
for diff, matchup in biggest_deficits[:5]:
    print(f"{matchup.week} {matchup.teams[0].name} {matchup.teams[1].name} - {diff}")

Biggest comebacks
8 b'Eric\xe2\x80\x99s a Bitch Team' b"Theodore's Poo Poo Hospital" - 58.3
1 b'Miami Steamrollers' b"eric's Awe-Inspiring Team" - 36.7
7 b'Miami Steamrollers' b"Cyrus's Cursed Team" - 35.2
8 b"Tim's Terrific Team" b"Cyrus's Cursed Team" - 31.5
11 b"Theodore's Poo Poo Hospital" b"Kevin's Incredible Team" - 23.6


In [ ]:
def get_next_game():
    

In [146]:
'''
Most/least average points after drop (team)
- Get all transactions
- Filter by drop
- Get NHL schedule
'''
transactions = query.get_league_transactions()

In [157]:
transactions_drop = [{"player_dropped": player, "timestamp": transaction.timestamp} for transaction in transactions for player in transaction.players if player.transaction_data.type == "drop"] # if player.transaction_data.type


In [ ]:
transactions_drop

In [ ]:
nhl_client.schedule.get_season_schedule(team_abbr="BUF", season="20232024")["games"]

In [ ]:
transactions

In [ ]:
[draft_result.player_key for draft_result in draft_results]

In [ ]:
'''
Biggest draft busts/steals
'''
# Get draft results
draft_results = query.get_league_draft_results()
draft_players = get_players([draft_result.player_key for draft_result in draft_results])
draft_players
draft_players_by_pos = {"F": [], "D": [], "G": []}
positions_map = {"C": "F", "LW": "F", "RW": "F", "D": "D", "G": "G"}
for draft_player in draft_players:
    draft_players_by_pos[positions_map[draft_player.primary_position]].append(draft_player)
# Get the top players by position
top_players_by_pos = {
    "F": get_top_n_players_by_position(len(draft_players_by_pos["F"]), "F"), 
    "D": get_top_n_players_by_position(len(draft_players_by_pos["D"]), "D"), 
    "G": get_top_n_players_by_position(len(draft_players_by_pos["G"]), "G")
}
# Get differences between draft player and top player
diffs = []
for key in draft_players_by_pos:
    for draft_player, top_player in zip(draft_players_by_pos[key], top_players_by_pos[key]):
        diff = round(draft_player.player_points.total - top_player.player_points.total, 1)
        diffs.append((diff, draft_player))
biggest_diffs = sorted(diffs, reverse=True, key=lambda x: x[0])
smallest_diffs = sorted(diffs, key=lambda x: x[0])
print("Biggest draft steals")
for diff, player in biggest_diffs[:5]:
    print(f"{player.name.first} {player.name.last} - {diff}")
print("Biggest draft busts")
for diff, player in smallest_diffs[:5]:
    print(f"{player.name.first} {player.name.last} - {diff}")